# Regression analysis

## 1. Import packages

In [2]:
import pandas as pd

import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

## 2. Load the data

In [3]:
data = pd.read_csv("../../data/reg_data_cz.csv")

In [7]:
data

,date,sentiment,disagreement,policy_change,policy_binned,hicp
0,1998-01-22,-0.589855,0.000000,0.00,0,12.1
1,1998-02-19,0.148933,0.000000,0.00,0,12.4
2,1998-03-19,0.690357,0.075000,0.00,0,12.4
3,1998-04-30,-0.957909,0.000000,0.00,0,12.0
4,1998-05-28,0.261321,0.000000,0.00,0,12.0
...,...,...,...,...,...,...
260,2024-03-20,-0.561032,0.014881,-0.50,-1,2.2
261,2024-05-02,0.326247,0.000000,-0.50,-1,3.1
262,2024-06-27,-0.093437,0.014881,-0.50,-1,2.2
263,2024-08-01,0.250993,0.000000,-0.25,-1,2.5


In [4]:
data["policy_binned"] = pd.Categorical(data["policy_binned"], ordered=True)

In [9]:
correlation = data[['sentiment', 'policy_change']].corr()

In [10]:
correlation

,sentiment,policy_change
sentiment,1.000000,0.545965
policy_change,0.545965,1.000000


## 3. Estimate the models

### Model 1

Model 1 is specified as:
$$
\Delta r_{t+1} = \beta_{0} + \beta_{1} \Delta r_{t} + \beta_{2} S_{t} + \epsilon_{t}
$$

- $\Delta r_{t+1}$: Policy rate change at $t+1$
- $\Delta r_{t}$: Policy rate change at $t$
- $S_{t}$: Sentiment at $t$
- $\epsilon_{t}$: Error term.

#### Prepare the dependent and independent variables

In [57]:
y = data["policy_binned"].cat.codes[1:].reset_index(drop=True)

In [58]:
X = pd.DataFrame({
    "policy_binned": data["policy_binned"].cat.codes[:-1],
    "sentiment": data["sentiment"][:-1],
})

In [59]:
model_1 = OrderedModel(y, X, distr='probit')

In [60]:
results_1 = model_1.fit()

Optimization terminated successfully.
         Current function value: 0.671835
         Iterations: 180
         Function evaluations: 304


In [61]:
print(results_1.summary())

                             OrderedModel Results                             
Dep. Variable:                      y   Log-Likelihood:                -177.36
Model:                   OrderedModel   AIC:                             362.7
Method:            Maximum Likelihood   BIC:                             377.0
Date:                Sat, 19 Oct 2024                                         
Time:                        16:15:26                                         
No. Observations:                 264                                         
Df Residuals:                     260                                         
Df Model:                           2                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
policy_binned     0.6818      0.180      3.792      0.000       0.329       1.034
sentiment         0.5420      0.125      4.

### Model 2

Model 2 is specified as:
$$
\Delta r_{t+1} = \beta_{0} + \beta_{1} \Delta r_{t} + \beta_{2} S_{t} + \beta_{3} Disag_{t} +\epsilon_{t}
$$

- $\Delta r_{t+1}$: Policy rate change at $t+1$.
- $\Delta r_{t}$: Policy rate change at $t$.
- $S_{t}$: Sentiment at $t$.
- $Disag_{t}$: Level of disagreement about policy rate change at $t$.
- $\epsilon_{t}$: Error term.

In [65]:
y = data["policy_binned"].cat.codes[1:].reset_index(drop=True)

In [66]:
X = pd.DataFrame({
    "policy_binned": data["policy_binned"].cat.codes[:-1],
    "sentiment": data["sentiment"][:-1],
    "disagreement": data["disagreement"][:-1],
})

In [67]:
model_2 = OrderedModel(y, X, distr='probit')

In [68]:
results_2 = model_2.fit()

/Users/bernhardbrunner/Desktop/venvs/finance_venv/lib/python3.11/site-packages/statsmodels/base/optimizer.py:737: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,
/Users/bernhardbrunner/Desktop/venvs/finance_venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [69]:
print(results_2.summary())

                             OrderedModel Results                             
Dep. Variable:                      y   Log-Likelihood:                -177.26
Model:                   OrderedModel   AIC:                             364.5
Method:            Maximum Likelihood   BIC:                             382.4
Date:                Sat, 19 Oct 2024                                         
Time:                        16:16:19                                         
No. Observations:                 264                                         
Df Residuals:                     259                                         
Df Model:                           3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
policy_binned     0.6685      0.182      3.670      0.000       0.311       1.025
sentiment         0.5442      0.125      4.

### Model 3

Model 3 is specified as:
$$
\Delta r_{t+1} = \beta_{0} + \beta_{1} \Delta r_{t} + \beta_{2} S_{t} + \beta_{3} (\pi_{t} - \pi^*) +\epsilon_{t}
$$

- $\Delta r_{t+1}$: Policy rate change at $t+1$.
- $\Delta r_{t}$: Policy rate change at $t$.
- $S_{t}$: Sentiment at $t$.
- $Disag_{t}$: Level of disagreement about policy rate change at $t$.
- $\epsilon_{t}$: Error term.